# D-Hydro Boezemmodel V4

## Introductie

Met deze notebook bouw je een boezemmodel voor Nooderzijlvest.

Zorg dat je beschikt over een werkende Python installatie en <a href="https://github.com/openearth/delft3dfmpy#installation">D-HYDAMO omgeving</a>. Het resulterende model moet geimporteerd kunnen worden in D-HYDRO versie 0.9.7.52006 of hoger.

De verwijzing naar bestanden is geschreven in het Nederlands. De rest van de code is geschreven in het Engels. Elk code-blok is voorzien van uitleg boven het desbetreffende blok, geschreven in het Nederlands.

Download de bestanden van <a href="https://drive.google.com/drive/folders/1BXNEwlEDCtShDj5KVuHQcukup-QLTuzR">Google Drive</a> en zet deze in de folder .\beheerregister

## Controle bestanden

Alle bestanden die gebruikt worden in deze tutorial staan in het code-blok hieronder. Wanneer je dit codeblok uitvoert wordt de aanwezigheid van deze bestanden gecontroleerd.



In [1]:
from pathlib import Path

beheerregister = Path(r".\beheerregister").absolute().resolve()

invoerbestanden = {"modelgebied": "Bemalings_en_lozingsgebieden.shp",
                   "branches": "Afaanvoervakken_DHYDRO_v5.shp",
                   "profielpunten": "profielpunten_3feb2021_actueel.shp",
                   "bruggen": "Bruggen_(doorstroomopeningen).shp",
                   "duikers": "duikers.shp",
                   "sifons": "sifons.shp",
                   "stuwen": "stuwen.shp",
                   "inlaten": "inlaten.shp",
                   "gemalen": "gemalen_v2.shp",
                   "peilgebieden": "Peilgebieden.shp"}

vorm_mapping = dict(rond=1,
                    driehoekig=1,
                    rechthoekig=3,
                    eivormig=1,
                    ellips=1,
                    Paraboolvormig=1,
                    trapeziumvormig=1,
                    heul=1,
                    muil=3,
                    langwerpig=1,
                    scherp=1,
                    onbekend=99,
                    overig=99)

for key, item in invoerbestanden.items():
    if not beheerregister.joinpath(item).exists():
        print(f"bestand voor {key} bestaat niet: {beheerregister.joinpath(item)}")

dimr_path = r"c:\Program Files (x86)\Deltares\D-HYDRO Suite 1D2D (Beta) (0.9.7.52006)\plugins\DeltaShell.Dimr\"kernels\x64\dimr\scripts\run_dimr.bat"


## Inlezen beheerregister in HyDAMO

### Aanmaken HyDAMO object

Alle benodigde modules worden geimporteerd en het HYDAMO object wordt aangemaakt bij het uitvoeren van onderstaand code-blok

In [2]:
from delft3dfmpy import DFlowFMModel, HyDAMO, Rectangular, DFlowFMWriter
from delft3dfmpy import DFlowRRModel, DFlowRRWriter
from delft3dfmpy.datamodels.common import ExtendedGeoDataFrame
import hydrotools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

hydamo = HyDAMO(extent_file=str(beheerregister.joinpath(invoerbestanden["modelgebied"])))

### Toevoegen waterlopen

De waterlopen (branches) worden toegevoegd met volgend code-blok:
* de branches worden toegekend aan het hydamo-object
* een HyDAMO ruwheidscode (4 = manning) wordt toegekend
* de eindpunten van de branches worden gesnapped binnen een tolerantie van 1m. De coordinaten worden ook afgerond op 1m.

In [3]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["branches"]),
                           "branches",
                           keep_columns=["HydroModel",
                                         "bodemhoogtebovenstrooms",
                                         "bodembreedte",
                                         "taludhellinglinkerzijde",
                                         "taludhellingrechterzijde",
                                         "IWS_W_WATB"],
                           column_mapping={"OVKIDENT": "code",
                                           "IWS_AVVHOB": "bodemhoogtebovenstrooms",
                                           "AVVBODDR": "bodembreedte",
                                           "AVVTALUL": "taludhellinglinkerzijde",
                                           "AVVTALUR": "taludhellingrechterzijde"}
                          )

hydamo.branches.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.branches["ruwheidstypecode"] = 4

hydamo.branches = hydrotools.snap_ends(hydamo.branches, tolerance=1, digits=1)

### Aanmaken principeprofielen
Als terugvaloptie voor branches zonder profielen schrijven we principe profielen weg die we later in deze notebook gebruiken.

In [4]:
principe_profielen_df = hydrotools.get_trapeziums(gdf,
                                                  "code",
                                                  "bodembreedte",
                                                  "bodemhoogtebovenstrooms",
                                                  "iws_w_watb",
                                                  "taludhellinglinkerzijde",
                                                  "taludhellingrechterzijde")

principe_profielen_df.to_excel(r".\xlsx\principe_profielen.xlsx")

### Toevoegen yz-profielen
De yz-profielen worden ingeladen:
* De profielpunten worden geopend
* de punten worden geordend en geconverteerd naar polylinen met een xyz coordinaten
* toekennen ruwheid (manning = 35)
* lijnen langer dan 500m worden weggegooid, omdat er soms kademuren PRO_TYPE = PRO hebben gekregen
* lijnen worden toegekend aan het HyDAMO object
* lijnen die niet snappen met de branches, worden weggegooid

ToDo:
* verbeteren toekenning ruwheid

In [5]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["profielpunten"]),
                           "crosssections",
                           attribute_filter={"osmomsch_1": ["WL", "Z1"],
                                             "PRO_TYPE": "PRO"},
                           column_mapping={"PROIDENT": "code",
                                           "IWS_VOLGNR": "order",
                                           "OSMOMSCH_1": "category",
                                           "iws_hoogte": "z"},
                           z_coord=True
                           )

grouper = gdf.groupby("code")
profiles = dict()

for code, prof_gdf in grouper:
    prof_gdf = prof_gdf.sort_values("order")  #Staan ook punten met zelfde order
    first_point = prof_gdf.iloc[0]["geometry"]
    cum_dist = -1
    line = []
    for idx, (_, row) in enumerate(prof_gdf.iterrows()):
        geom = row["geometry"]
        distance = geom.distance(first_point)
        if distance > cum_dist:
            cum_dist = distance
            line += [(geom.x, geom.y, row["z"])]

    profiles[code] = [code, LineString(line)]

profiles_gdf = gpd.GeoDataFrame.from_dict(profiles,
                                          orient="index",
                                          columns=["code",
                                                   "geometry"]
                                          )

profiles_gdf["ruwheidstypecode"] = 4
profiles_gdf["ruwheidswaarde"] = 35
profiles_gdf["codegerelateerdobject"] = None

profiles_gdf = profiles_gdf.loc[profiles_gdf["geometry"].length < 500]

hydamo.crosssections.set_data(profiles_gdf,
                              index_col="code",
                              check_columns=True,
                              check_geotype=True)

hydamo.crosssections.snap_to_branch(hydamo.branches, snap_method="intersecting")
hydamo.crosssections.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen geparameteriseerde profielen

Per brug moet er een profiel worden toegevoegd

In [6]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "parametrised_profiles",
                           column_mapping={"bodemh_bov": "bodemhoogtebovenstrooms",
                                           "bodemh_ben": "bodemhoogtebenedenstrooms",
                                           "doorstroom": "bodembreedte",
                                           "hoogte_ond": "hoogteinsteeklinkerzijde",
                                           })

grouper = gdf.groupby("code")
data = {item: [] for item in ["code",
                              "bodemhoogtebovenstrooms",
                              "bodemhoogtebenedenstrooms",
                              "bodembreedte",
                              "hoogteinsteeklinkerzijde",
                              "geometry",
                              "codegerelateerdobject"]}

for code, df in grouper:
    data["code"] += [f"PRO_{code}"]
    data["codegerelateerdobject"] += [code]
    for item in ["bodemhoogtebovenstrooms",
                 "bodemhoogtebenedenstrooms",
                 "hoogteinsteeklinkerzijde",
                 "geometry"]:
        data[item] += [df.iloc[0][item]]
    data["bodembreedte"] += [df['bodembreedte'].sum()]

gdf = gpd.GeoDataFrame(data)
gdf["geometry"] = gdf.apply((lambda x: LineString([[x["geometry"].x-0.1,
                                                    x["geometry"].y-0.1],
                                                  [x["geometry"].x+0.1,
                                                   x["geometry"].y+0.1]])),
                            axis=1)

gdf["hoogteinsteekrechterzijde"] = gdf["hoogteinsteeklinkerzijde"]
gdf["ruwheidswaarde"] = 15
gdf["ruwheidstypecode"] = 4
gdf["taludhellinglinkerzijde"] = 1
gdf["taludhellingrechterzijde"] = 1

hydamo.parametrised_profiles.set_data(gdf,
                                      index_col="code",
                                      check_columns=True,
                                      check_geotype=False)

### Toevoegen bruggen
Per brug moet er een profiel worden toegevoegd

In [7]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["bruggen"]),
                           "bridges",
                           column_mapping={"doorstro_2": "lengte",
                                           "bodemh_bov": "hoogteonderzijde"})

grouper = gdf.groupby("code")
data = {"code": [code for code, frame in grouper],
        "hoogteonderzijde": [frame["hoogteonderzijde"].values[0] for code,
                             frame in grouper],
        "lengte": [frame["lengte"].values[0] for code, frame in grouper],
        "geometry": [frame["geometry"].values[0] for code, frame in grouper]
        }
gdf = gpd.GeoDataFrame(data)

gdf["hoogtebovenzijde"] = gdf["hoogteonderzijde"] + 1

gdf["dwarsprofielcode"] = gdf["code"].apply(lambda x: f"PRO_{x}")
gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidstypecode"] = 4
gdf["ruwheidswaarde"] = 70

hydamo.bridges.set_data(gdf, index_col="code", check_columns=True, check_geotype=False)
hydamo.bridges.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.bridges.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen duikers en siphons
duikers en sifons worden als cuverts aan het HyDAMO object toegekend

ToDo
* vorm moet naar HyDAMO codering worden omgeschreven

In [8]:
culverts_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["duikers"]),
                                    "culverts",
                                    column_mapping={"hoogte_a1": "hoogteopening",
                                                    "breedte": "breedteopening",
                                                    "bok_benede": "hoogtebinnenonderkantbenedenstrooms",
                                                    "bok_boven": "hoogtebinnenonderkantbovenstrooms",
                                                    "vorm": "vormcode"})

siphons_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["sifons"]),
                                   "culverts",
                                   column_mapping={"hoogte": "hoogteopening",
                                                   "breedte": "breedteopening",
                                                   "bok_benede":"hoogtebinnenonderkantbenedenstrooms",
                                                   "bok_boven":"hoogtebinnenonderkantbovenstrooms",
                                                   "vorm":"vormcode"})

gdf = gpd.GeoDataFrame(pd.concat([culverts_gdf,siphons_gdf], ignore_index=True))

gdf.loc[gdf['vormcode'].isnull(), 'vormcode'] = 'onbekend'
gdf.loc[:, 'vormcode'] = gdf.apply((lambda x: vorm_mapping[x['vormcode'].lower()]), axis=1)

gdf["intreeverlies"] = 0.7
gdf["uittreeverlies"] = 0.7
gdf["ruwheidswaarde"] = 70
gdf["ruwheidstypecode"] = 4

hydamo.culverts.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.culverts.snap_to_branch(hydamo.branches, snap_method="ends", maxdist=5)
hydamo.culverts.dropna(axis=0, inplace=True, subset=["branch_offset"])

hydamo.culverts.loc[hydamo.culverts["lengte"].isna(), "lengte"] = 20

### Toevoegen stuwen

In [9]:
gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["stuwen"]),
                           "weirs",
                           column_mapping={"kstident": "code",
                                           "kstsoort": "soortstuwcode",
                                           "kstkrubr": "laagstedoorstroombreedte",
                                           "kstmikho": "laagstedoorstroomhoogte",
                                           "kstregel": "soortregelbaarheidcode"})

gdf["afvoercoefficient"] = 1

hydamo.weirs.set_data(gdf, index_col="code", check_columns=True, check_geotype=True)
hydamo.weirs.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=10)
hydamo.weirs.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen gemalen

In [10]:
pumps_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["gemalen"]),
                                 "pumps",
                                 column_mapping={"gecom_max_": "maximalecapaciteit"})

gemalen_gdf = pumps_gdf.copy()
pumps_gdf["codegerelateerdobject"] = pumps_gdf["code"].copy()
pumps_gdf["code"] = [f"PMP_{code}" for code in pumps_gdf["code"]]
hydamo.gemalen.set_data(gemalen_gdf,
                        index_col="code",
                        check_columns=True,
                        check_geotype=True)
hydamo.gemalen.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.gemalen.dropna(axis=0, inplace=True, subset=["branch_offset"])
hydamo.pumps.set_data(pumps_gdf,
                      index_col="code",
                      check_columns=True,
                      check_geotype=True)
hydamo.pumps.snap_to_branch(hydamo.branches, snap_method="overal", maxdist=5)
hydamo.pumps.dropna(axis=0, inplace=True, subset=["branch_offset"])

### Toevoegen afsluitmiddelen

In [11]:
gdf = pd.DataFrame({"code": [],
                    "soortafsluitmiddelcode": [],
                    "codegerelateerdobject": []})

hydamo.afsluitmiddel.set_data(gdf, index_col="code")

### Toevoegen sturing

In [12]:
gpg_gdf = hydrotools.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]),
                                 "sturing",
                                 column_mapping={"GPGIDENT": "code",
                                                 "OPVAFWZP": "streefwaarde"
                                                 })

gpg_gdf["codegerelateerdobject"] = [code.replace("GPG", "") for code in gpg_gdf["code"]]
gpg_gdf = gpg_gdf[gpg_gdf["codegerelateerdobject"].isin(hydamo.gemalen["code"])]
con_gdf = hydamo.gemalen[~hydamo.gemalen["code"].isin(gpg_gdf["codegerelateerdobject"])]
con_gdf = gpd.GeoDataFrame(data={"code": con_gdf["code"].values,
                                 "geometry": con_gdf["geometry"].values})
con_gdf["codegerelateerdobject"] = con_gdf["code"]
con_gdf["streefwaarde"] = -999

gdf = gpd.GeoDataFrame(pd.concat([gpg_gdf, con_gdf], ignore_index=True))

gdf["bovenmarge"] = gdf["streefwaarde"] + 0.05
gdf["ondermarge"] = gdf["streefwaarde"] - 0.05
gdf["doelvariabelecode"] = 1

hydamo.sturing.set_data(gdf, index_col="code")

### Filteren

Alle branches die niet bij "Boezemmodel_v4" gooien we uit het model. Idem voor alle kunstwerken + profielen die naar deze branches zijn gesnapped. Om het resultaat te beoordelen exporteren we alles naar shape-files. We slaan het hydamo object ook op als "pickle", zodat we bovenstaande stappen niet elke keer hoeven te herhalen

In [13]:
hydamo = hydrotools.filter_model(hydamo, attribute_filter={"HydroModel": "Boezemmodel_v4"})
hydrotools.export_shapes(hydamo, path=Path(r"./hydamo_shp/boezemmodel_v4"))
hydrotools.save_model(hydamo, file_name=Path(r"./hydamo_model/boezemmodel_v4.pickle"))


## Converteren naar DFM

### Aanmaken dfm-klasse

In [14]:
dfmmodel = DFlowFMModel()

### Inlezen kunstwerken

In [15]:
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

2021-02-25 13:59:03,331 - delft3dfmpy.converters.hydamo_to_dflowfm - hydamo_to_dflowfm - INFO - Currently only simple weirs can be applied. From Hydamo the attributes 'laagstedoorstroomhoogte' and 'kruinbreedte' are used to define the weir dimensions.


### Aanmaken 1d netwerk

In [16]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=100.0, seperate_structures=True)

2021-02-25 13:59:06,572 - delft3dfmpy.core.dfm - dfm - WARNING - Some structures are not linked to a branch.
2021-02-25 13:59:06,580 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000010 at: [0.0, 691.94, 1642.4933902054406], due to the structures at [-0.001, 671.552, 712.328, 1163.135, 1539.892, 1642.4943902054406].
2021-02-25 13:59:06,580 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000015 at: [0.0, 106.271, 141.85887653227576], due to the structures at [-0.001, 78.108, 134.434, 141.85987653227576].
2021-02-25 13:59:06,584 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000017 at: [0.0, 506.58549999999997, 522.864, 971.4883654003871], due to the structures at [-0.001, 25.63, 498.27, 514.901, 530.827, 971.4893654003871].
2021-02-25 13:59:06,587 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000030 at: [0.0, 2.5425, 325.6167677365957], due to the structures at [-0.001, 0.1, 4.985, 325.61776

2021-02-25 13:59:06,711 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000615 at: [0.0, 460.5815, 463.840805478718], due to the structures at [-0.001, 144.349, 322.782, 457.424, 463.739, 463.84180547871796].
2021-02-25 13:59:06,712 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000617 at: [0.0, 240.4535, 593.9086377149054], due to the structures at [-0.001, 217.858, 263.049, 390.768, 593.9096377149053].
2021-02-25 13:59:06,717 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000642 at: [0.0, 22.7795, 376.1764634617298], due to the structures at [-0.001, 15.491, 30.068, 376.17746346172976].
2021-02-25 13:59:06,718 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF000656 at: [0.0, 2.069, 204.186, 210.8305, 214.50377727101835], due to the structures at [-0.001, 0.1, 4.038, 200.09, 208.282, 213.379, 214.50477727101836].
2021-02-25 13:59:06,720 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes

2021-02-25 13:59:06,791 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001131 at: [0.0, 35.701499999999996, 347.4619213986779], due to the structures at [-0.001, 0.1, 71.303, 140.912, 347.36192139867785, 347.46292139867785].
2021-02-25 13:59:06,797 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001148 at: [0.0, 6.5035, 63.089999999999996, 122.9315, 144.7265, 159.0447845704711], due to the structures at [-0.001, 0.1, 12.907, 113.273, 132.59, 156.863, 159.0457845704711].
2021-02-25 13:59:06,797 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001172 at: [0.0, 12.597999999999999, 377.65748483996896], due to the structures at [-0.001, 3.223, 21.973, 114.856, 236.713, 377.65848483996893].
2021-02-25 13:59:06,797 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001174 at: [0.0, 389.105, 444.9528056427896], due to the structures at [-0.001, 0.1, 341.065, 437.145, 444.9538056427896].
2021-02-25 13:59:06

2021-02-25 13:59:06,867 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001590 at: [0.0, 354.43, 667.6843306263461, 682.7796612526923], due to the structures at [-0.001, 10.987, 334.892, 373.968, 492.37, 652.689, 682.6796612526923, 682.7806612526923].
2021-02-25 13:59:06,869 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001600 at: [0.0, 271.1685, 1123.769535563856], due to the structures at [-0.001, 240.622, 301.715, 482.629, 1123.770535563856].
2021-02-25 13:59:06,870 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001624 at: [0.0, 542.7294999999999, 1521.7820804604353], due to the structures at [-0.001, 537.731, 547.728, 1512.852, 1521.7830804604353].
2021-02-25 13:59:06,873 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF001654 at: [0.0, 4.208, 379.1346054243062], due to the structures at [-0.001, 1.188, 7.228, 379.13560542430616].
2021-02-25 13:59:06,876 - delft3dfmpy.core.dfm - dfm - INFO

2021-02-25 13:59:06,977 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002221 at: [0.0, 6.302, 16.1775, 22.5645, 180.7996291477335], due to the structures at [-0.001, 0.1, 12.504, 19.851, 25.278, 180.6996291477335, 180.8006291477335].
2021-02-25 13:59:06,977 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002236 at: [0.0, 409.8995, 455.097191363579], due to the structures at [-0.001, 395.691, 424.108, 455.09819136357896].
2021-02-25 13:59:06,977 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002237 at: [0.0, 75.3905, 148.15300000000002, 171.71250890444963], due to the structures at [-0.001, 16.535, 134.246, 162.06, 171.71350890444964].
2021-02-25 13:59:06,977 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF002241 at: [0.0, 19.16, 484.5978363251505], due to the structures at [-0.001, 9.204, 29.116, 226.826, 484.5988363251505].
2021-02-25 13:59:06,977 - delft3dfmpy.core.dfm - dfm - INFO - Added 

...

2021-02-25 13:59:07,394 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004625 at: [0.0, 64.814, 136.5365940203473], due to the structures at [-0.001, 0.1, 129.528, 136.5375940203473].
2021-02-25 13:59:07,397 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004658 at: [0.0, 72.958, 144.15658696002328], due to the structures at [-0.001, 21.173, 124.743, 144.15758696002328].
2021-02-25 13:59:07,399 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004673 at: [0.0, 3.6965, 22.227235545630304], due to the structures at [-0.001, 0.1, 7.293, 22.228235545630305].
2021-02-25 13:59:07,400 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004676 at: [0.0, 2.421, 7.2705, 255.91243908344418], due to the structures at [-0.001, 0.1, 4.742, 9.799, 242.245, 255.9134390834442].
2021-02-25 13:59:07,404 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF004717 at: [0.0, 340.501, 565.3661873179884], 

2021-02-25 13:59:07,480 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005166 at: [0.0, 10.5015, 504.1753576205113], due to the structures at [-0.001, 7.428, 13.575, 488.851, 504.1763576205113].
2021-02-25 13:59:07,484 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005171 at: [0.0, 26.928, 50.931, 163.16780104897566], due to the structures at [-0.001, 6.897, 46.959, 54.903, 163.16880104897567].
2021-02-25 13:59:07,486 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005181 at: [0.0, 16.0745, 234.93076821167512], due to the structures at [-0.001, 0.1, 32.049, 228.838, 234.93176821167512].
2021-02-25 13:59:07,490 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005185 at: [0.0, 28.313, 52.512, 55.557, 60.368, 63.723, 100.1565, 169.978, 206.733, 212.3755, 215.337, 254.00631793341142, 292.75563586682284], due to the structures at [-0.001, 5.037, 51.589, 53.435, 57.679, 63.057, 64.389, 135.924, 204.0

2021-02-25 13:59:07,568 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005669 at: [0.0, 3.6504999999999996, 10.4975, 15.028, 87.19046767302581], due to the structures at [-0.001, 0.1, 7.201, 13.794, 16.262, 87.19146767302581].
2021-02-25 13:59:07,568 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005671 at: [0.0, 2.3225, 39.472, 150.853703965116], due to the structures at [-0.001, 0.1, 4.545, 74.399, 150.85470396511602].
2021-02-25 13:59:07,576 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005682 at: [0.0, 5.3265, 23.938000000000002, 138.59940700717692], due to the structures at [-0.001, 0.1, 10.553, 37.323, 138.60040700717693].
2021-02-25 13:59:07,578 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF005687 at: [0.0, 5.752, 29.60000000000582], due to the structures at [-0.001, 0.1, 11.404, 29.601000000005822].
2021-02-25 13:59:07,580 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes o

2021-02-25 13:59:07,687 - delft3dfmpy.core.dfm - dfm - WARNING - Structures KDU010709, KST6542 have the same location.
2021-02-25 13:59:07,688 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006477 at: [0.0, 6.1259999999999994, 325.5624251041951], due to the structures at [-0.001, 0.1, 12.152, 208.665, 318.047, 325.56342510419506].
2021-02-25 13:59:07,690 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006482 at: [0.0, 42.942499999999995, 55.7841855106377], due to the structures at [-0.001, 36.507, 49.378, 55.785185510637696].
2021-02-25 13:59:07,695 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF006543 at: [0.0, 10.787, 31.506999999999998, 51.381, 75.1235, 99.49549999999999, 120.26849999999999, 140.9395, 161.497, 192.22693382828552], due to the structures at [-0.001, 0.1, 21.474, 41.54, 61.222, 89.025, 109.966, 130.571, 151.308, 171.686, 192.22793382828553].
2021-02-25 13:59:07,700 - delft3dfmpy.core.dfm - dfm - INF

2021-02-25 13:59:07,794 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010577 at: [0.0, 25.038, 91.83785281968377], due to the structures at [-0.001, 6.966, 43.11, 91.83885281968378].
2021-02-25 13:59:07,797 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010586 at: [0.0, 69.2065, 78.82300000000001, 86.94257733502819], due to the structures at [-0.001, 62.449, 75.964, 81.682, 86.9435773350282].
2021-02-25 13:59:07,798 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010588 at: [0.0, 77.172, 128.3048114435671], due to the structures at [-0.001, 56.76, 97.584, 128.3058114435671].
2021-02-25 13:59:07,802 - delft3dfmpy.core.dfm - dfm - INFO - Added 1d mesh nodes on branch OAF010592 at: [0.0, 1690.692, 2356.06, 2933.2555, 3148.8395, 3189.783018731833], due to the structures at [-0.001, 7.776, 256.501, 892.678, 1180.169, 1689.164, 1692.22, 1798.283, 2353.942, 2358.178, 2575.527, 2904.388, 2962.123, 3139.74, 3157.939, 3189.7

### Toevoegen cross-sections

We kennen profielen toe, waar we profielen missen vullen we deze aan met principeprofielen.

In [17]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} branches are still missing a cross section.")
print(f"{len(dfmmodel.crosssections.get_structures_without_crosssection())} structures are still missing a cross section.")

if len(dfmmodel.crosssections.get_branches_without_crosssection()) > 0:
    print("adding trapezium profiles on branches with missing crosssections.")
    dfmmodel = hydrotools.add_trapeziums(dfmmodel, principe_profielen_df)

2021-02-25 13:59:09,791 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of branches without cross section is: 2385.
2021-02-25 13:59:17,422 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 867.
No parametrised crossections available for branches.
2021-02-25 13:59:17,430 - delft3dfmpy.io.dfmreader - dfmreader - INFO - Before adding the number of structures without cross section is: 634.
2021-02-25 13:59:17,456 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'dwarsprofielen' the number of branches without cross section is: 634.
2021-02-25 13:59:17,504 - delft3dfmpy.io.dfmreader - dfmreader - INFO - After adding 'normgeparametriseerd' the number of structures without cross section is: 0.
867 branches are still missing a cross section.
0 structures are still missing a cross section.
adding trapezium profiles on branches with missing crosssections.


### Wegschrijven model

In [19]:
dfmmodel.mdu_parameters["refdate"] = 20000101
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 144.0 * 1 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "120. 0. 0."
dfmmodel.mdu_parameters["cflmax"] = 0.7

dfmmodel.dimr_path = dimr_path
fm_writer = DFlowFMWriter(dfmmodel, output_dir=r"modellen\boezemmodel", name="boezemmodel")

fm_writer.objects_to_ldb()
fm_writer.write_all()

if Path(r"modellen\boezemmodel\fm\boezemmodel.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

Model is weggeschreven


## Importeren in de D-Hydro suite

Open het model nu in D-Hydro:
1. Open een "Empty Project"
2. In de "Home" ribbon, ga naar "Import" en selecteer "Flow Flexible Mesh Model"
3. Selecteer het bestand "boezemmodel.mdu" in ".\dfm_model\fm\boezemmodel.mdu"
4. Wacht tot het model is geimporteerd.....

Het geimporteerde model is nu zichtbaar in de D-Hydro suite
<img src="png/boezemmodel.png"/>